# Implementación de t-SNE en Python

**Contenido:**

- Mostraremos cómo aplicar t-SNE a un conjunto de datos utilizando Scikit-learn.
- Visualizaremos los resultados para comprender la estructura de los datos.

## Código de Ejemplo - Importación y Preparación de Datos


In [ ]:

# Importar librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE


**Explicación:**
- Utilizamos el conjunto de datos de dígitos escritos a mano disponible en Scikit-learn.
- Este conjunto de datos es adecuado para t-SNE debido a su alta dimensionalidad (64 características) y múltiples clases (10 dígitos).

## Por qué aplicar la t de Student

En t-SNE (t-distributed Stochastic Neighbor Embedding), la **distribución t de Student** se utiliza para calcular la matriz de similitud en el **espacio reducido (baja dimensión)** principalmente por dos razones fundamentales:

1. Colas más largas que la Gaussiana:

- En el espacio original (alta dimensión), las similitudes entre puntos se modelan con una distribución Gaussiana, que tiene colas cortas.
- Pero si se usara una Gaussiana también en el espacio reducido, los puntos distantes (que deberían tener similitud baja) no podrían alejarse lo suficiente, y esto causaría un efecto de "amontonamiento" (crowding problem).
- La distribución t de Student con 1 grado de libertad (equivalente a una Cauchy) tiene colas mucho más largas, permitiendo que los puntos disímiles se separen mejor en el espacio reducido.

2. Mejor preservación de relaciones locales:

- El objetivo de t-SNE es preservar las relaciones locales: que si dos puntos eran muy similares en el espacio original, también lo sean en el reducido.
- La distribución t favorece esta preservación al permitir una fuerte penalización cuando puntos similares se representan como lejanos, pero también permite que puntos no similares estén muy separados sin distorsionar demasiado la función de costo (KL divergencia).

In [ ]:
from scipy.stats import norm, t

x = np.linspace(-5, 5, 500)
gaussian = norm.pdf(x, loc=0, scale=1)
student_t = t.pdf(x, df=1)  # t de Student con 1 grado de libertad (colas largas)

plt.plot(x, gaussian, label='Gaussiana', linewidth=2)
plt.plot(x, student_t, label='t de Student (df=1)', linewidth=2)
plt.title("Comparación de distribuciones")
plt.xlabel("Distancia")
plt.ylabel("Densidad de probabilidad")
plt.legend()
plt.grid(True)
plt.show()

## Comparación de las matrices de similitud

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.datasets import load_digits
from scipy.spatial.distance import pdist, squareform

# Cargamos los primeros 100 datos para hacerlo manejable
X, y = load_digits(return_X_y=True)
X = X[:100]
y = y[:100]

# Ejecutamos t-SNE
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=1000, random_state=42)
Y = tsne.fit_transform(X)

# --- 1. Matriz de similitud P usando Gaussiana ---
def compute_p_matrix(X, perplexity=30.0):
    # Distancias al cuadrado
    distances = pairwise_distances(X, squared=True)
    n = X.shape[0]
    P = np.zeros((n, n))
    for i in range(n):
        beta = 1.0
        Di = distances[i, np.arange(n) != i]
        H_target = np.log(perplexity)
        
        # Usamos búsqueda binaria para encontrar beta adecuada
        betamin, betamax = None, None
        for _ in range(50):
            Pi = np.exp(-Di * beta)
            Pi = Pi / Pi.sum()
            H = -np.sum(Pi * np.log(Pi + 1e-10))
            H_diff = H - H_target
            if np.abs(H_diff) < 1e-5:
                break
            if H_diff > 0:
                betamin = beta
                beta = beta * 2 if betamax is None else (beta + betamax) / 2
            else:
                betamax = beta
                beta = beta / 2 if betamin is None else (beta + betamin) / 2
        Pi_full = np.zeros(n)
        Pi_full[np.arange(n) != i] = Pi
        P[i] = Pi_full
    # Simetrizamos
    P = (P + P.T) / (2 * n)
    return P

# --- 2. Matriz de similitud Q usando t de Student ---
def compute_q_matrix(Y):
    distances = squareform(pdist(Y, 'sqeuclidean'))
    Q = 1 / (1 + distances)
    np.fill_diagonal(Q, 0)
    Q = Q / Q.sum()
    return Q

# Calculamos P y Q
P = compute_p_matrix(X, perplexity=30)
Q = compute_q_matrix(Y)

# --- 3. Visualización ---
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].imshow(P, cmap='viridis')
axs[0].set_title("Similitud P (Gaussiana en alta dimensión)")
axs[0].set_xlabel("Punto j")
axs[0].set_ylabel("Punto i")

axs[1].imshow(Q, cmap='viridis')
axs[1].set_title("Similitud Q (t de Student en baja dimensión)")
axs[1].set_xlabel("Punto j")
axs[1].set_ylabel("Punto i")

plt.tight_layout()
plt.show()


## Aplicación de t-SNE

In [ ]:

# Cargar el conjunto de datos
digits = load_digits()
X = digits.data  # Datos de entrada
y = digits.target  # Etiquetas

# Mostrar dimensiones de los datos
print(f"Dimensiones de X: {X.shape}")
print(f"Dimensiones de y: {y.shape}")

# Configurar t-SNE
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=1000, random_state=42)

# Aplicar t-SNE para reducir la dimensionalidad a 2D
X_embedded = tsne.fit_transform(X)

# Mostrar dimensiones de los datos transformados
print(f"Dimensiones de X_embedded: {X_embedded.shape}")


**Explicación:**
- Configuramos t-SNE para reducir los datos a 2 dimensiones.
- Establecemos perplexity=30, learning_rate=200 y n_iter=1000 como valores típicos.
- Aplicamos fit_transform para obtener los datos en el espacio reducido.

## Visualización de los Resultados


In [ ]:
# Convert target names to a list of strings
labels = [str(label) for label in digits.target_names]

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y, cmap='tab10', alpha=0.7)

# Get handles and labels from the scatter plot
handles, _ = scatter.legend_elements()

# Use the custom labels
plt.legend(handles=handles, labels=labels, title="Dígitos", bbox_to_anchor=(1.05, 1), loc='upper left')

# Add titles and labels
plt.title('Visualización de Dígitos utilizando t-SNE')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.grid(True)
plt.show()



**Explicación:**
- Creamos una gráfica de dispersión donde cada punto representa una imagen de un dígito.
- Los puntos se colorean según su etiqueta real.
- Observamos cómo t-SNE agrupa puntos similares juntos, facilitando la identificación de clusters correspondientes a cada dígito.

## Interpretación de la Visualización
**Análisis de Resultados:**
- Los dígitos similares están agrupados juntos.
- Algunos clusters están claramente separados, indicando que t-SNE pudo capturar la estructura de los datos.
- Puntos intermedios pueden representar dígitos difíciles de clasificar o con características compartidas.